***GENERATED CODE FOR 2010kneigb PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '17', 'mean': '', 'stddev': '', 'min': 'Accountants and auditors', 'max': 'Wholesale and retail buyers, except farm products', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation'}, {'feature_label': 'Occupation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '1306.94', 'stddev': '268.7', 'min': '1020', 'max': 'Na', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_weekly'}, {'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import pyspark


def Knearestneighborregressor(sparkDF, features, labels, TrainingPercent=100):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    clf = KNeighborsRegressor()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    clf.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % clf.score(X_test, y_test))

    data = {'model': clf,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 2010kneigbHooks.ipynb
try:
	#sourcePreExecutionHook()

	predicthighestincomesample = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Predict_Highest_Income_Sample.csv', 'filename': 'Predict_Highest_Income_Sample.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 2010kneigbHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(predicthighestincomesample,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Occupation", "transformation_label": "String Indexer"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "Accountants and auditors", "max": "Wholesale and retail buyers, except farm products", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "M_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "187.06", "stddev": "145.19", "min": "44", "max": "618", "missing": "0"}, "updatedLabel": "M_workers"}, {"transformationsData": [{"feature_label": "M_weekly", "transformation_label": "String Indexer"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "1306.94", "stddev": "268.7", "min": "1020", "max": "Na", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "242.53", "stddev": "210.83", "min": "63", "max": "846", "missing": "0"}, "updatedLabel": "F_workers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_weekly", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "1023.76", "stddev": "166.39", "min": "680", "max": "1348", "missing": "0"}, "updatedLabel": "F_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "All_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "429.71", "stddev": "341.76", "min": "106", "max": "1464", "missing": "0"}, "updatedLabel": "All_workers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "All_weekly", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "1140.35", "stddev": "204.98", "min": "742", "max": "1431", "missing": "0"}, "updatedLabel": "All_weekly"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 2010kneigbHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=Knearestneighborregressor(autofe, ["M_workers", "F_workers", "F_weekly", "All_workers", "All_weekly", "Occupation_stringindexer"], "M_weekly_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
